In [1]:
import cudf
from pyproj import Transformer, CRS

In [2]:
def read_state(state):
    print('reading '+state, end='\r')
    df = cudf.read_csv('../data/census_data/population_%s.csv'%(state), usecols=['GISJOIN','x', 'y'])
    df.GISJOIN = df.GISJOIN.astype('int')
    inProj = '+proj=aea +lat_1=29.5 +lat_2=45.5 +lat_0=37.5 +lon_0=-96 +x_0=0 +y_0=0 +datum=NAD83 +units=m +no_defs' # Latitude and longitudes
    outProj = 'epsg:4326' # 2D projected points
    transformer = Transformer.from_crs(inProj, outProj, always_xy=True)
    df['x'], df['y'] = transformer.transform(df['x'].to_array(), df['y'].to_array()) # Apply transformation
    print('completed',end='\r')
    return df

In [3]:
list_of_states = ['AL', 'AK', 'AZ', 'AR', 'CO', 'CT', 'DE', 'DC', 'FL', 'CA',
                  'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME',
                  'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH',
                  'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI',
                  'SC', 'SD', 'TN', 'TX', 'UT', 'VA','VT', 'WA', 'WI', 'WV', 'WY', 'PR']

In [4]:
df = [read_state(x) for x in list_of_states]

In [11]:
#leave PR
df = df[:-1]

In [12]:
final_df = cudf.concat(df)
del(df)

In [17]:
final_df = final_df.reset_index()
final_df.drop_column('index')

In [20]:
final_df['p_id'] = final_df.index.astype('int32')
final_df

,GISJOIN,x,y,p_id
0,1000100211002002,-9.662807e+06,3.831871e+06,0
1,1000100211002002,-9.663283e+06,3.832983e+06,1
2,1000100211002002,-9.663042e+06,3.832885e+06,2
3,1000100211002002,-9.662848e+06,3.833558e+06,3
4,1000100211002002,-9.662436e+06,3.833098e+06,4
...,...,...,...,...
308745533,56004509511002304,-1.162458e+07,5.461661e+06,308745533
308745534,56004509511002304,-1.162415e+07,5.461604e+06,308745534
308745535,56004509511002304,-1.162405e+07,5.461376e+06,308745535
308745536,56004509511002304,-1.162429e+07,5.461773e+06,308745536


In [21]:
final_df.dtypes

GISJOIN      int64
x          float64
y          float64
p_id         int32
dtype: object

In [22]:
final_df.GISJOIN.unique()

0           1000100201001000
1           1000100201001003
2           1000100201001005
3           1000100201001007
4           1000100201001009
                 ...        
2805556    56004509513003096
2805557    56004509513003104
2805558    56004509513003112
2805559    56004509513003120
2805560    56004509513003128
Name: GISJOIN, Length: 2805561, dtype: int64

In [23]:
final_df.to_parquet('../data/all_statas_us_lat_long.parquet')

/home/ajay/anaconda3/envs/rapids_dev_0.12/lib/python3.6/site-packages/cudf/io/parquet.py:70: UserWarning: Using CPU via PyArrow to write Parquet dataset, this will be GPU accelerated in the future
  "Using CPU via PyArrow to write Parquet dataset, this will "


In [13]:
final_df.drop_column('GISJOIN')

In [18]:
write_arrow_to_disk(final_df.to_arrow().to_batches()[0], 'all_states_us_lat_long_no_gisjoin')

saving as an arrow file on the path specified: all_states_us_lat_long_no_gisjoin
